# Assignment 5: Implementation of Maps

<html>
<div class="alert alert-info" role="alert" style="margin-top: 10px">
In this exercise you should implement the drift, dipole, quadrupole (+thin), solenoid, combined function magnet and fringe field maps and apply them to a Gaussian beam. Plot the phase spaces.
</div>
</html>

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>If you use google colab, run this cell:</strong>
    </div>
</html>

In [ ]:
# for google colab, run this cell
!git clone https://github.com/potato18z/pam1-hs2021.git
import sys
sys.path.append('./pam1-hs2021')

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
<strong>If you run it locally, run</strong>
               </div>
</html>

```bash
$ cd .../pam1-hs2021
...pam1-hs2021$ git pull
```
<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
to get the updated repository.</div></html>

# Define a bunch

In order to build maps we need some properties that are globally available. Those are the Lorentz factor $\gamma$, the particle mass $m\ [MeV/c^2]$ and the particle charge $q\ [e]$.
Those parameters are given in ```Parameter.py``` and initialized by
```Python
import AcceLEGOrator.Parameter as param

param.gamma0 = # ...
param.mass = # ...
param.charge = # ...
```
After importing the `Physics` module you can use the function
```
Physics.getGamma(ekin, epot)
```
to obtain the Lorentz factor.

We have already define a `Proton` class in ```Particle.py```:
```Python
class Proton(Particle):
    
    def __init__(self):
        super(Proton, self).__init__(Constants.pmass,
                                     1.0,
                                     'Proton')
```
Instantiante a proton species using the `Proton` class and set all global variables from its attributes.

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Create a proton species.</li>
        </ul>
    </div>
</html>

In [ ]:
from AcceLEGOrator import Proton
# TODO: instantiate proton object
# you can refer to the Introduction of pyaccelerator notebook

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Set global variables with given kinetic energy $E_{kin}=100 MeV$.</li>
        </ul>
    </div>
</html>

In [ ]:
import AcceLEGOrator.Parameter as param
from AcceLEGOrator import Physics, Constants

ekin = 100 # MeV

# TODO: Define the parameters using the particle's properties

param.mass    = ... # MeV / c^2
param.charge  = ... # e
param.gamma_0 = Physics.getGamma(...)
print ( 'Gamma = ', param.gamma_0)

Now you can use the `Gaussian` class to create a Gaussian bunch.

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Create a Gaussain bunch like in Introduction_to_pyAcceLEGOrator.ipynb.</li>
        </ul>
    </div>
</html>

In [ ]:
# TODO:
# 1. create distribution

# mean
mu = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

# covariance matrix
C = np.array([[ 16.0, 0.0,  0,  0,  0, 0],
              [ 0.0, 1.0,  0,  0,  0, 0],
              [ 0,  0, 16.0,  0.0,  0, 0],
              [ 0,  0, 0.0, 1,  0, 0],
              [ 0,  0,  0,  0, 0.709, 0.0],
              [ 0,  0,  0,  0,  0.0, 0.0981]])

# 2. create bunch

If you need to copy a bunch without referencing to the same object, run
```python
from copy import deepcopy
bunch1 = deepcopy(bunch)
```
Check inside `Bunch.py` for more details.

In [1]:
from copy import deepcopy

# Maps
All linear maps are derived classes of the base class `Map`. Implement the necessary member functions.
```Python
class Map(object):
    
    # param R its 6x6 matrix
    # param length of element [m]
    def __init__(self, R, length):
        self.R = np.matrix(R)
        self.length = length
    
    # Print map properties, like its type,
    # magnetic field strength, length, etc.
    def __str__(self):
        return '\n'
    
    # return a new instance with specific length
    @abstractmethod
    def get(self, length):
        pass
```
You can find more detail about the usage of `super()` in [this answer](https://stackoverflow.com/a/576183).

If you need any global variable for the construction of the 6x6 matrix, call them by e.g. ```param.gamma_0```.
Apply the maps to the bunch by multiplication
```Python
M = Drift(1.0)
bunch.particles =  M * bunch.particles
```

Then plot the initial and final phase space use the given function `plot_phase_space`. If you want to show them on the same plot with different color, change the function as you need.

In [ ]:
from matplotlib import pyplot as plt
## Change the function as you wish if you want to plot more, add color and legends etc.

# param ax is the axis to plot on
# param xvar phase space variable in x-dir
# param yvar phase space variable in y-dir
# param xlab label for x-axis
# param ylab label for y-axis
def plot_phase_space(ax, xvar, yvar, xlab, ylab):
    vmin = min(min(xvar), min(yvar))
    vmin += 0.1 * vmin
    vmax = max(max(xvar), max(yvar))
    vmax += 0.1 * vmax
    
    ax.set_xlim(vmin, vmax)
    ax.set_ylim(vmin, vmax)
    ax.set_xlabel(xlab)
    ax.set_ylabel(ylab)
    ax.scatter(xvar, yvar, s=1)

## Drift
<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Implement the drift map as given in the lecture slides.</li>
        </ul>
    </div>
</html>

In [ ]:
from AcceLEGOrator import Map

class Drift(Map):
    
    # param length in [m]
    def __init__(self, length):
        
        # TODO 
        
        R = np.matrix(# ... )
        
        super(Drift, self).__init__(R, length)
    
    def __str__(self):
        # TODO
    
    def get(self, length):
        # TODO

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Apply the map on the created bunch. Plot initial and final phase space.</li>
        </ul>
    </div>
</html>

In [ ]:
fig = plt.figure(dpi=300)
fig.set_size_inches(9,9)
axis = fig.add_subplot(111)

# TODO

## Dipole
<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Implement the dipole map as given in the lecture slides.</li>
        </ul>
    </div>
</html>

In [ ]:
class Dipole(Map):
    
    # -----------------------------------------------------
    # param length specified in [m]
    # param b0 is the magnetic field strength [T]
    # -----------------------------------------------------
    def __init__(self, length, b0):
        # TODO
    
    
    def __str__(self):
        # TODO
    
    
    def get(self, length):
        # TODO

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Apply the map on the created bunch. Plot initial and final phase space.</li>
        </ul>
    </div>
</html>

In [ ]:
# TODO
Di = Dipole(length=1.0, b0 = 10.0)

## Quadrupole
<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Implement the quadrupole map as given in the lecture slides.</li>
        </ul>
    </div>
</html>

In [ ]:
class Quadrupole(Map):
    # -----------------------------------------------------
    # param length specified in [m]
    # param gradB is the magnetic field gradient in [T/m] (b2/r0)
    # -----------------------------------------------------
    def __init__(self, length, gradB):
        # TODO
    
    
    def __str__(self):
        # TODO
    
    
    def get(self, length):
        # TODO

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Apply the map on the created bunch. Plot initial and final phase space.</li>
        </ul>
    </div>
</html>

In [ ]:
# TODO
Q = Quadrupole(length=1.0, gradB = 0.5)

## Thin Quadrupole
<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Implement the thin lens approximation of the quadrupole as given in the lecture slides.</li>
        </ul>
    </div>
</html>

In [ ]:
class ThinQuadrupole(Map):
    
    # param f is the focal length
    def __init__(self, f):
        # TODO
    
    def __str__(self):
        # TODO
    
    def get(self, length):
        # TODO

Apply the map on the created bunch. Plot initial and final phase space.

In [ ]:
# TODO
TQ = ThinQuadrupole(f = 1.5)

# Solenoid
<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Implement the solenoid map as given in the lecture slides.</li>
        </ul>
    </div>
</html>

In [ ]:
class Solenoid(Map):
    # -----------------------------------------------------
    # param length specified in [m]
    # param b0 is the magnetic field strength [T]
    # -----------------------------------------------------
    def __init__(self, length, b0):
        # TODO
    
    
    def __str__(self):
        # TODO
    
    
    def get(self, length):
        # TODO

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Apply the map on the created bunch. Plot initial and final phase space.</li>
        </ul>
    </div>
</html>

In [ ]:
# TODO
S = Solenoid(length=1.0, b0 = 10.0)

# Combined function magnet
<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Implement the combined function magnet map as given in the lecture slides.</li>
        </ul>
    </div>
</html>

In [ ]:
class Combined(Map):
    # -----------------------------------------------------
    # param length specified in [m]
    # param b0 is the magnetic field in [T]
    # param gradB is the magnetic field gradient in [T/m]
    # -----------------------------------------------------
    def __init__(self, length, b0, gradB):
        # TODO
    
    
    def __str__(self):
        # TODO
    
    
    def get(self, length):
        # TODO

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Apply the map on the created bunch. Plot initial and final phase space.</li>
        </ul>
    </div>
</html>

In [ ]:
# TODO
C = Combined(length=1.0, b0 = 10.0, gradB = 0.5)

# Fringe Field
<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Implement the fringe field map as given in the lecture slides.</li>
        </ul>
    </div>
</html>

In [ ]:
class Fringe(Map):
    
    # -----------------------------------------------------
    # param b0 is the magnetic field strength [T]
    # param angle [deg]
    # -----------------------------------------------------
    def __init__(self, b0, angle):
        # TODO
    
    
    def __str__(self):
        # TODO
    
    
    def get(self, length):
        # TODO

<html>
    <div class="alert alert-info" style="background-color:rgba(255, 0, 0, 0.6);
                                         margin-top:10px;
                                         color:white;
                                         border-color:rgba(255, 0, 0, 0.3)">
        <strong>TODO:</strong>
        <ul>
            <li>Apply a dipole with fringe field on the created bunch. Plot initial and final phase space.</li>
        </ul>
    </div>
</html>

In [ ]:
# TODO
# apply the dipole
D = Dipole(length=1.0, b0 = 10)
# apply the fringe field
F = Fringe(b0 = 10, angle = 20)